In [ ]:
import numpy as np
from numpy import linalg
from scipy.special import binom
from typing import Callable, List

In [ ]:
IntIndexing = Callable[[int], int]
FloatIndexing = Callable[[int], np.float64]
FloatDoubleIndexing = Callable[[int, int], np.float64]

Row = List[np.float64]
Rows = List[List[np.float64]]
ListOfMatrices = List[np.ndarray]
ListOfListOfMatrices = List[ListOfMatrices]

## 1. Math in the text

The inputs are a number $n$ of points and a number $k$ of zero moments. In chapter 5, section "Software Limitations" the text explains why it assumes a relation between $n$ and $k$: $n = 2^l k$ for some $l$, so $\displaystyle l = \log_2\left(\frac{n}{k}\right)$.

## 1.1. Table of symbols and values in text and in the implementation

|Symbol  |Text definition                                                       |Code implementation                                  |
|--------|----------------------------------------------------------------------|-----------------------------------------------------|
|l       |$\displaystyle \log_2\left(\frac{n}{k}\right)$                        |$\displaystyle \log_2\left(\frac{n}{k}\right)$       |
|j       |$2, \dots, l$                                                         |$1, \dots, l - 1$                                    |
|i       |$\displaystyle 1, \dots, \frac{n}{2^j k}$                             |$\displaystyle 0, \dots, \frac{n}{2^{(j + 1)} k} - 1$|
|$\mu$   |$\displaystyle \mu_{j,i} = \frac{x_{1 + (i - 1)k2^j} + x_{ik2^j}}{2}$ |                                                     |
|$\sigma$|$\displaystyle \sigma_{j,i} = \frac{x_{ik2^j} - x_{1+(i - 1)k2^j}}{2}$|                                                     |
|$s$     |$s_i = (i - 1)2k$                                                     |$s_i = i2k$                                          |

In [ ]:
def s_builder(k: int) -> IntIndexing:
  '''s'''
  def s(i) -> int:
    return i*2*k
  return  s


def µ_builder(x: np.ndarray, k: int) -> FloatDoubleIndexing:
  def µ(j: int, i: int) -> np.float64:
    idx = ( i*k*2**(j + 1), (i + 1)*k*2**(j + 1) - 1 )
    return (x[ idx[0] ] + x[ idx[1] ]) / 2
  return µ


def σ_builder(x: np.ndarray, k: int) -> FloatDoubleIndexing:
  def σ(j: int, i: int) -> np.float64:
    idx = ( (i + 1)*k*2**(j + 1) - 1, i*k*2**(j + 1) )
    return (x[ idx[0] ] - x[ idx[1] ]) / 2
  return σ


def µ_rows(x: np.ndarray, k: int) -> Rows:
  n = len(x) ; l = int(np.log2(n // k))
  return [
    [
      (x[i*k*2**(j + 1)] + x[(i + 1)*k*2**(j + 1) - 1]) / 2
      for i in range(n // (k*2**(j + 1)))
    ]
    for j in range(l)
  ]


def σ_rows(x: np.ndarray, k: int) -> Rows:
  n = len(x) ; l = int(np.log2(n // k))
  return [
    [
      (x[(i + 1)*k*2**(j + 1) - 1] - x[i*k*2**(j + 1)]) / 2
      for i in range(n // (k*2**(j + 1)))
    ]
    for j in range(l)
  ]


def s_list(n: int, k: int) -> Row:
  return [i*2*k for i in range(n // (2*k))]

# Step 1

## 1.1. Math in the text

Compute $M'_{1,i}$ for $\displaystyle i= 1, \dots, \frac{n}{2k}$

$$
\begin{equation} \tag{4.12}
  M'_{1,i} = \begin{pmatrix} \displaystyle
  1      & \frac{x_{s_{i + 1}} + \mu_{1,i}}{\sigma_{1,i}} & \dots & \left(\frac{x_{s_{i + 1}} + \mu_{1,i}}{\sigma_{1,i}}\right)^{2k - 1} \\
  1      & \frac{x_{s_{i + 2}} + \mu_{1,i}}{\sigma_{1,i}} & \dots & \left(\frac{x_{s_{i + 2}} + \mu_{1,i}}{\sigma_{1,i}}\right)^{2k - 1} \\
  \vdots & & & \vdots \\
  1      & \frac{x_{s_{i + 2k}} + \mu_{1,i}}{\sigma_{1,i}} & \dots & \left(\frac{x_{s_{i + 2k}} + \mu_{1,i}}{\sigma_{1,i}}\right)^{2k - 1} \\
  \end{pmatrix}
\end{equation}
$$

## 2. Math for the code

Values for the code:

$$
\displaystyle
i = 0, \dots, \frac{n}{2k} - 1 \\
s_i =  
$$



In [ ]:
def Ms_first_row(x: np.ndarray, k: int, l: int) -> ListOfMatrices:
  '''the first row of the matrix of shifted and scaled matrices
it's what in the text is called M\'1,i '''
  n = len(x) ; µ = µ_builder(x, k) ; σ = σ_builder(x, k) ; s = s_builder(k)
  return [
    np.array(
        
      [
        [
          ( (x[s(i) + row - 1] - µ(0, i)) / σ(0, i) )**col
          for col in range(2*k)
        ]
        for row in range(2*k)
      ], dtype=np.float64
    )
    for i in range(n//(2*k))
  ]


def M_1s_list(
    x: np.ndarray,
    k: int,
    µ: Rows,
    σ: Rows,
    s: List[int]
) -> ListOfListOfMatrices:
  return [
    np.array([
      [
        ((x[s[i] + row - 1] - µ[0][i]) / σ[0][i])**col
        for col in range(2*k)
      ]
      for row in range(2*k)
    ], dtype=np.float64)
    for i in range(n // (2*k))
  ]

In [ ]:
n, k = 8, 2
l = int(np.log2(n // k))
x = np.linspace(0, 1, num=n, endpoint=True)
# M_1s = Ms_first_row(x, k, l)
µ = µ_rows(x, k)
σ = σ_rows(x, k)
s = s_list(n, k)
M_1s = M_1s_list(x, k, µ, σ, s)
# M_1s

In [ ]:
M_1s_list(x, k, µ, σ, s)

[array([[ 1.00000000e+00,  3.66666667e+00,  1.34444444e+01,
          4.92962963e+01],
        [ 1.00000000e+00, -1.00000000e+00,  1.00000000e+00,
         -1.00000000e+00],
        [ 1.00000000e+00, -3.33333333e-01,  1.11111111e-01,
         -3.70370370e-02],
        [ 1.00000000e+00,  3.33333333e-01,  1.11111111e-01,
          3.70370370e-02]]),
 array([[ 1.        , -1.66666667,  2.77777778, -4.62962963],
        [ 1.        , -1.        ,  1.        , -1.        ],
        [ 1.        , -0.33333333,  0.11111111, -0.03703704],
        [ 1.        ,  0.33333333,  0.11111111,  0.03703704]])]

In [ ]:
Ms_first_row(x, k, l)

[array([[ 1.00000000e+00,  3.66666667e+00,  1.34444444e+01,
          4.92962963e+01],
        [ 1.00000000e+00, -1.00000000e+00,  1.00000000e+00,
         -1.00000000e+00],
        [ 1.00000000e+00, -3.33333333e-01,  1.11111111e-01,
         -3.70370370e-02],
        [ 1.00000000e+00,  3.33333333e-01,  1.11111111e-01,
          3.70370370e-02]]),
 array([[ 1.        , -1.66666667,  2.77777778, -4.62962963],
        [ 1.        , -1.        ,  1.        , -1.        ],
        [ 1.        , -0.33333333,  0.11111111, -0.03703704],
        [ 1.        ,  0.33333333,  0.11111111,  0.03703704]])]

# Step 2

In [ ]:
def Us_first_row(M_1s) -> RowOfMatrices:
  '''the first row of the matrix of shifted and scaled matrices
it's what in the text is called U\'1,i '''
  return [linalg.qr(M_1)[0].T for M_1 in M_1s]


U_1s_list = Us_first_row

In [ ]:
U_1s = U_1s_list(M_1s)

In [ ]:
[U_1_i.T @ U_1_i for U_1_i in U_1s]

[array([[ 1.00000000e+00, -2.10766226e-16, -2.91335815e-17,
          4.18457652e-17],
        [-2.10766226e-16,  1.00000000e+00,  2.43492568e-17,
          2.44437163e-17],
        [-2.91335815e-17,  2.43492568e-17,  1.00000000e+00,
         -3.02521951e-17],
        [ 4.18457652e-17,  2.44437163e-17, -3.02521951e-17,
          1.00000000e+00]]),
 array([[ 1.00000000e+00, -1.49666486e-16,  4.45046514e-17,
         -1.56773574e-16],
        [-1.49666486e-16,  1.00000000e+00, -2.83504380e-18,
          1.02946037e-16],
        [ 4.45046514e-17, -2.83504380e-18,  1.00000000e+00,
          5.77269489e-17],
        [-1.56773574e-16,  1.02946037e-16,  5.77269489e-17,
          1.00000000e+00]])]

# Step 3

In [ ]:
def S_array(k: int, µ: np.float64, σ: np.float64) -> np.ndarray:
  return np.array(
    [
      [binom(j, i)*(-µ)**(j-i)/σ**j for j in range(2*k)]
      for i in range(2*k)
    ]
  )


def S1(n: int, k: int, µ: RowsOfMatrices, σ: RowsOfMatrices) -> RowsOfMatrices:
  l = int(np.log2(n // k))
  return [
    [
      S_array(
        k,
        µ=(µ[j][i] - µ[j - 1][2*i])/σ[j - 1][2*i],
        σ=σ[j][i]/σ[j - 1][2*i]
      )
      for i in range(n // (k*2**(j + 1)))
    ]
    for j in range(1, l)
  ]


def S2(n: int, k: int, µ: RowsOfMatrices, σ: RowsOfMatrices) -> RowsOfMatrices:
  l = int(np.log2(n // k))
  return [
    [
      S_array(
          k,
          µ=(µ[j][i] - µ[j - 1][2*i + 1])/σ[j - 1][2*i + 1],
          σ=σ[j][i]/σ[j - 1][2*i + 1]
      )
      for i in range(n // (k*2**(j + 1)))
    ]
    for j in range(1, l)
  ]